In [38]:
class Node:
    def __init__(self, key):
        self.key = key
        self.terms = set()
        self.childs = dict()

In [39]:
class Tree:
    def __init__(self) -> None:
        self.root = Node('')
    
    def populate_tree(self, terms_list):
        for term in terms_list:
            self.insert_term(term, term, self.root)
    
    def insert_term(self, key, term, node=None):
        if node is None:
            return self.insert_term(key, term, self.root)

        if not key:
            node.terms.add(term)
            return
        if key[0] in node.childs:
            self.insert_term(key[1:], term, node.childs[key[0]])
        else:
            node.childs[key[0]] = Node(key[0])
            self.insert_term(key[1:], term, node.childs[key[0]])
    
    def visual_tree(self, node=None):
        # print(type(node))
        if node is None:
            self.visual_tree(self.root)
            return
        # print(node.key, node.terms)
        for child in node.childs.values():
            self.visual_tree(child)
    
    def find_terms(self, term, node=None):
        if node is None:
            # print('node is None')
            return self.find_terms(term, self.root)
        
        if not term:
            # print('empty term\tnode.key=', node.key)
            result = node.terms
            for child in node.childs.values():
                result.update(self.find_terms(term, child))
            return result
        
        # print('term=', term, '\tnode.key=', node.key)
        if term[0] not in node.childs:
            return set()
        return self.find_terms(term[1:], node.childs[term[0]])


In [40]:
class TranspositionIndex:
    def __init__(self):
        self.index = Tree()
    
    def transpose_term(self, term):
        term += '$'
        transponsed = [term[i:] + term[:i] for i in range(len(term))]
        # print(transponsed)
        return transponsed
    
    def populate_index(self, term_list):
        for term in term_list:
            for el in self.transpose_term(term):
                self.index.insert_term(el, term)
    
    def show_index(self):
        self.index.visual_tree()
    
    def find_term(self, term):
        term += '$'

        if '*' in term:
            spl = term.split('*')
            result = self.index.find_terms(spl[-1]+spl[0])
        else:
            result = self.index.find_terms(term)
        return result

In [41]:
class ThreeGramIndex:
    def __init__(self):
        self.index = Tree()
    
    def get_grammes(self, term, k=3):
        length = len(term)
        grammes = [term[i:i+k]for i in range(length-k+1)]
        # print(grammes)
        return grammes
    
    def populate_index(self, term_list):
        for term in term_list:
            for el in self.get_grammes('$' + term + '$'):
                self.index.insert_term(el, term)
    
    def show_index(self):
        self.index.visual_tree()
    
    def find_term(self, term):
        term = '$' + term + '$'
        result = None

        if '*' in term:
            spl = term.split('*')
            for el in spl:
                if len(el)<3:
                    continue
                for g in self.get_grammes(el):
                    if result is None:
                        result = self.index.find_terms(g)
                    else:
                        result &= self.index.find_terms(g)
        else:
            for g in self.get_grammes(term):
                if result is None:
                    result = self.index.find_terms(g)
                else:
                    result &= self.index.find_terms(g)
        
        return result

In [42]:
class SuffixTree:
    def __init__(self) -> None:
        self.index = Tree()
        self.reverse_index = Tree()
    
    def populate_index(self, term_list):
        for term in term_list:
            self.index.insert_term(term, term)
            self.reverse_index.insert_term(term[::-1], term)
    
    def show_index(self):
        print('tree')
        self.index.visual_tree()
        print('reverse tree')
        self.reverse_index.visual_tree()
    
    def find_term(self, term):
        if '*' in term:
            spl = term.split('*')
            result = self.index.find_terms(spl[0]) & self.reverse_index.find_terms(spl[-1][::-1])
        else:
            result = self.index.find_terms(term) & self.reverse_index.find_terms(term[::-1])
        
        return result

In [43]:
import codecs
import re
import os

tr_index = TranspositionIndex()
thrg_index = ThreeGramIndex()
suf_tree = SuffixTree()
dictionary = set()

data_folder_path = os.path.join('.', 'data')
books_list = os.listdir(data_folder_path)

for book in books_list:
    book_path = os.path.join(data_folder_path, book)
    with codecs.open(book_path, "r", "utf_8_sig") as fileObj:
        text = fileObj.read()

    words = re.findall("[a-z]+[’'-]?[a-z]+", text.lower())
    words = set(words)
    dictionary.update(words)
    
    tr_index.populate_index(words)
    thrg_index.populate_index(words)
    suf_tree.populate_index(words)

In [44]:
def visual_set(sett, k=14):
    if not sett:
        return
    i = k+1
    for el in sett:
        i -= 1
        if not i: break 
        print(el, end=', ')

In [45]:
visual_set(dictionary)

irregularities, re-transformed, holly, indiscreet, guelder-rose, helicopters, meat-pies, iron-gray, bag, famed, quiz, increases, blotting-book, counterpanes, 

In [46]:
def test_structure(struct):
    terms = (
        'persons',
        '*ersons',
        'per*ons',
        'person*',
        '*erson*',
        'pe*s*s',
        '*e*s',
        'p*o*',
    )
    for t in terms:
        res = struct.find_term(t)
        print(t, end='\t{')
        visual_set(res)
        print('}')

In [47]:
print('\t--- \t Перестановочний індекс \t ---\t')
test_structure(tr_index)
print('\n\t--- \t Триграмний індекс \t ---\t')
test_structure(thrg_index)
print('\n\t--- \t Суфіксне дерево \t ---\t')
test_structure(suf_tree)


	--- 	 Перестановочний індекс 	 ---	
persons	{persons, unpersons, }
*ersons	{persons, unpersons, }
per*ons	{persons, persecutions, percussions, perambulations, perfections, persuasions, perceptions, perversions, personifications, }
person*	{persons, personne, personalities, personally, personable, persona, person’s, personal, personified, personated, personality, personage, personifications, personification, }
*erson*	{wrathfully, irregularities, re-transformed, holly, landfall, enamelled, indiscreet, waiting, stair-rails, guelder-rose, clump, helicopters, trio, neighboring, }
pe*s*s	{perkins, peddlers, penurious, perambulations, persists, penitentials, peculiarities, pennies, perversions, pens, pettishness, petrov’s, petitioners, perils, }
*e*s	{quinns, tops, george’s, savors, relatives, irregularities, points, suffers, grates, dismissals, supporters, griffins, partnerships, stair-rails, }
p*o*	{preyed, points, paroxysm, pause, partnerships, penseur, phantoms, painted, pearl-like, pil

In [95]:
import re

pattern = 'pe*s*s'
res = tr_index.find_term(pattern)
print(res)
filtered_res = [t for t in res if re.match(pattern.replace('*', '.*'), t)]
print(filtered_res)


{'petites', 'performances', 'petitioners', 'peignoirs', 'peterson’s', 'pears', 'pencils', 'peewits', 'pers', 'pegs', 'perhaps', 'peer’s', 'perversions', 'pews', 'perishes', 'peeress', 'perfumer’s', 'personalities', 'pervious', 'peers', 'pets', 'peaches', 'perceives', 'petrovs', 'petrovna’s', 'pebble-stones', 'pecks', 'penalties', 'pertinacious', 'peelings', 'pettishness', 'petritsky’s', 'per’aps', 'persons', 'personifications', 'personages', 'perplexities', 'perrin’s', 'pearls', 'petitions', 'pennies', 'perfidious', 'peters', 'performs', 'perceptions', 'periodicals', 'perfections', 'peasant’s', 'perilous', 'petrov’s', 'peacemakers', 'pertains', 'petticoats', 'persecutors', 'pebbles', 'penknives', 'peter’s', 'perils', 'periods', 'pedant’s', 'peculiarities', 'petits', 'petitioner’s', 'pewter-dishes', 'petals', 'petersburg’s', 'perkins', 'person’s', 'pedlars', 'peddles', 'peals', 'pelisses', 'penurious', 'permits', 'pedestals', 'perambulations', 'persuades', 'percussions', 'peoples', 'pen